## RNN
This model is a simple 3 layer RNN followed by a fully connected layer. We found the GRUs worked better than LSTMs, besides resulting in faster train times. We arrived at these parameters through [hyperparameter-tuning](../hyperparameter-tuning.ipynb).

![Network](rnn.png)

In [1]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, GRU, \
    concatenate, Add, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
from IPython.display import Image

### Import data

In [2]:
df_1 = pd.read_csv("../data/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../data/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

### Load model

In [18]:
model = load_model("rnn_ptbdb.h5")

In [6]:
img = plot_model(model, to_file="rnn.png", show_shapes=True)

### Or ALTERNATIVELY train it

In [3]:
seq_len = 187

def get_model():
    n_class = 1
    inp = Input(shape=(187, 1))
    x = GRU(128,
           return_sequences=True,
           dropout=0.2)(inp)
    x = GRU(128,
           return_sequences=True,
           dropout=0.2)(x)
    x = GRU(128,
           return_sequences=False,
           dropout=0.2)(x)
    
    dense = Dense(64, activation='relu')(x)
    dense = Dropout(0.2)(dense)
    output = Dense(n_class, activation='sigmoid')(dense)
    
    opt = tensorflow.keras.optimizers.Adam(lr=0.001)
    
    model = models.Model(inputs=inp, outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

In [6]:
model = get_model()
file_path = "rnn_ptbdb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000
10476/10476 - 330s - loss: 0.5961 - accuracy: 0.7225 - val_loss: 0.6113 - val_accuracy: 0.6996
Epoch 2/1000
10476/10476 - 441s - loss: 0.5923 - accuracy: 0.7244 - val_loss: 0.6133 - val_accuracy: 0.6996
Epoch 3/1000
10476/10476 - 536s - loss: 0.5908 - accuracy: 0.7244 - val_loss: 0.6129 - val_accuracy: 0.6996
Epoch 4/1000


KeyboardInterrupt: 

### Evaluation

In [5]:
pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

auc_roc = roc_auc_score(Y_test, pred_test)

print("AUROC score : %s "% acc)

precision, recall, _ = precision_recall_curve(Y_test, pred_test)

auc_prc = auc(recall, precision)
print("AUPRC score : %s "% auc_prc)

print(confusion_matrix(Y_test, pred_test))

Test f1 score : 0.22223882639341913 
Test accuracy score : 0.278941944349021 
AUROC score : 0.278941944349021 
AUPRC score : 0.6445123937218665 
[[ 799   10]
 [2089   13]]
